In [1]:
import os
import cv2
import numpy as np

In [ ]:
'''单模板单目标匹配'''
'''简单情况：原始图像有不同目标，模板匹配一个'''
background = cv2.imread(os.getcwd()+"\\images\\background.jpg")
temp1 = cv2.imread(os.getcwd()+"\\images\\template.png")
height , width , c = temp1.shape # 行对应高
result = cv2.matchTemplate(background,temp1,cv2.TM_SQDIFF_NORMED)
# templ 模板图像
# method的方法
# cv2.TM_SQDIFF =0 平方差匹配 匹配程度越高计算结果越小 完全匹配的结果为0
# cv2.TM_SQDIFF_NORMED =1 标准平方差匹配
# cv2.TM_CCORR =2 相关匹配，匹配程度越高计算结果越大
# cv2.TM_CCORR_NORMED =3 标准相关匹配
# cv2.TM_CCOEFF =4 相关系数匹配 计算结果为-1~1之间的浮点数，1表示完全匹配，0表示毫无关系，-1表示亮度刚好相反
# cv2.TM_CCOEFF_NORMED # 标准相关系数匹配
# mask 只对cv2.TM_SQDIFF和cv2.TM_CCORR_NORMED支持
minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(result)
target_left_up_x ,target_left_up_y = minLoc # 最匹配的像素左上角坐标，匹配系数minVal
target_rigth_down_x,target_rigth_down_y =target_left_up_x + width  ,target_left_up_y + height
target = (target_rigth_down_x,target_rigth_down_y)# 右下角坐标
print(f"target(x,y)=({target_left_up_x},{target_left_up_y}) minVal={minVal}")
# minMaxLoc(src[, mask]) -> minVal, maxVal, minLoc, maxLoc
# 返回的minVal为数组的最小值，maxVal最大值，对于平方差匹配找到minVal所处的索引坐标（x，y）minLoc，其他则是找到最大的坐标maxLoc
cv2.rectangle(background,minLoc,target,(0,0,255),2)
cv2.imshow('background',background)
cv2.imshow('template',temp1)
cv2.waitKey()
cv2.destroyAllWindows()

target(x,y)=(331,379) minVal=0.00010066070535685867


In [26]:
'''eg1：对比情况：2个相同目标的原始图像，模板匹配最好的'''
template = cv2.imread(os.getcwd()+"\\images\\warningSign.jpg")
sign1 = cv2.imread(os.getcwd()+"\\images\\warningSign1.jpg")
sign2 = cv2.imread(os.getcwd()+"\\images\\warningSign2.jpg")
lst = []
for i in [sign1,sign2]:
    result = cv2.matchTemplate(i,template,cv2.TM_SQDIFF_NORMED)
    print(result,result.shape,(i.shape[0]-template.shape[0]+1,i.shape[1]-template.shape[1]+1)) # W-w+1/H-h+1
    # print(result.any())
    lst.append((np.mean(result),np.var(result)))
    print("\n")
print(lst) # 比较匹配结果的均值和方差，显然第1个更匹配，事实上也是更匹配

[[0.38154617 0.38201162 0.38246095 ... 1.         1.         1.        ]
 [0.38246876 0.3829326  0.38339502 ... 1.         1.         1.        ]
 [0.38340285 0.38386446 0.38433874 ... 1.         1.         1.        ]
 ...
 [0.3751112  0.37644726 0.37782952 ... 0.39742836 0.39644328 0.39548334]
 [0.37416846 0.37551406 0.3769138  ... 0.396822   0.39586458 0.39490384]
 [0.37324706 0.37458292 0.37597787 ... 0.3960817  0.39514837 0.3941699 ]] (451, 410) (451, 410)


[[0.36223736 0.36225468 0.36225897 ... 1.         1.         1.        ]
 [0.36141506 0.36143646 0.3614439  ... 1.         1.         1.        ]
 [0.36058593 0.36060953 0.36062354 ... 1.         1.         1.        ]
 ...
 [0.40099543 0.40264174 0.40429923 ... 0.7776425  0.7753703  0.7730494 ]
 [0.40201414 0.40365487 0.40531093 ... 0.7776382  0.77544826 0.7732217 ]
 [0.40303096 0.4046648  0.4063131  ... 0.7776284  0.77551645 0.7733851 ]] (2675, 3642) (2675, 3642)


[(0.5066021, 0.014943199), (0.7361401, 0.0739)]


In [99]:
'''eg2：不同格式的图像文件查找重复的图片'''
root = r"C:\\Users\\chenb\\jupyterLab\\images\\test\\"
rescale = (100,100) # width,height
image_paths = os.listdir(root)
img_lst = []
corr_lst = [] # 相关系数存放
match_lst = [] # 所有匹配对
match_idx = [] # 匹配系数满足条件的匹配对
for image_path in image_paths:
    path = root + image_path
    #print(path)
    im = cv2.imread(path)
    im = cv2.resize(im,rescale)
    img_lst.append(im)
for j in range(len(img_lst)-1):
    # print(j)
    current_template = img_lst[j] #当前模板图像依次等于0,1,2...8
    idx = j 
    for i in img_lst[j+1:]:
        idx += 1 
        #print(f"模板图像：{j+1}.jpg,匹配图像：{idx+1}.jpg") # 检验i的循环次数是否符合预期
        match_lst.append((j+1,idx+1)) # 符合习惯，第1个图像是0.jpg
        result = cv2.matchTemplate(i,current_template,cv2.TM_CCOEFF_NORMED)
        if j==3 and idx==9 : # 第4个图像和第10个图像的匹配结果
            print(result)
        corr_lst.append(result[0])
# print(len(corr_lst))
assert len(corr_lst) == np.cumsum(np.arange(10))[-1] == len(match_lst) # 1+2+3..+9 ,累积匹配了45次
for idx ,value in enumerate(corr_lst):
    #print(value)
    if value > 0.5:
       match_idx.append(match_lst[idx])
match_idx

[[-0.22887117]]


[(2, 5), (2, 8), (3, 6), (3, 10), (5, 8), (6, 10)]

In [96]:
'''单模板多目标匹配'''
background = cv2.imread(os.getcwd()+"\\images\\background2.jpg")
temp = cv2.imread(os.getcwd()+"\\images\\template.png")
height , width , c = temp.shape # 行对应高
result = cv2.matchTemplate(background,temp,cv2.TM_CCOEFF_NORMED)
for row in range(result.shape[0]): # Height
    for col in range(result.shape[1]): # Width
        if result[row,col] > 0.99 :
            cv2.rectangle(background,(col,row),(col+width,height+row),(0,0,255),2)
cv2.imshow('im',background)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
'''多模板多目标匹配，只需要进行多次单模板多目标匹配即可'''